In [1]:
import pandas as pd
from pulp import *
import pickle

In [2]:
### filepaths
primary_facility_path = '../Data/Primary_Facility_50.csv'
secondary_facility_path = '../Data/Secondary_Facility_500.csv'
supplier_path = '../Data/Supplier.csv'
demand_path = '../ResearchData/1.Demand_Distribution.csv'
distance_path = '../Data/DISTANCE_DICT_VP50_VQ_500.pkl'

In [3]:
# fetch input data
p_df = pd.read_csv(primary_facility_path)
q_df = pd.read_csv(secondary_facility_path)
s_df = pd.read_csv(supplier_path)
demand_df = pd.read_csv(demand_path)

q_df['Index'] = q_df.index
s_df['Index'] = s_df.index
p_df['Index'] = p_df.index
demand_df['Index'] = demand_df.index


p_df['Index'] = p_df['Index'].apply(lambda x : 'P'+ str(x))
q_df['Index'] = q_df['Index'].apply(lambda x : 'Q'+ str(x))
s_df['Index'] = s_df['Index'].apply(lambda x : 'S'+ str(x))
demand_df['Index'] = demand_df['Index'].apply(lambda x : 'K' + str(x))

with open(distance_path, 'rb') as fp:
    DISTANCE_DICT = pickle.load(fp) 

In [4]:
## SETS
PRIMARY_FACILITY = tuple(p_df['Index'])
SECONDARY_FACILITY =  tuple(q_df['Index'])
CUSTOMER = tuple(demand_df['Index'])
SUPPLIER = tuple(s_df['Index'])


In [5]:
## VARIABLES
SUPPLIER_TO_PRIMARY = 'Xsp'
PRIMARY_TO_SECONDARY = 'Ypq'
PRIMARY_TO_PRIMARY = 'Tpp'
SECONDARY_TO_CUSTOMER = 'Zqk'
FACILITY = 'G'

In [6]:
## PARAMETERS
DEMAND = tuple(demand_df['Demand'])
DISTANCE = DISTANCE_DICT

FIXED_COST_PRIMARY = 250000
FIXED_COST_SECONDARY = 25000
PACKAGING_COST_PRIMARY = 0.035
PACKAGING_COST_SECONDARY = 0.140

R_PRIMARY = 2
R_SECONDARY = 50
C_SUPPLIER = 0.0006
C_PRIMARY = 0.0024
C_CUSTOMER = 0.012


In [7]:
DEMAND_DICT = dict(zip(CUSTOMER, DEMAND))

In [8]:
# # model
# model = grb.Model("MILP")

# Create a model
model = LpProblem("MILP", sense=LpMinimize)  # You can use LpMaximize for maximization problems


In [9]:
## Define decision variables
L = 0.5
# L = LpVariable(name="L", lowBound=0, cat=LpContinuous)
Xsp_var = LpVariable.dicts(SUPPLIER_TO_PRIMARY, [(s, p) for s in SUPPLIER for p in PRIMARY_FACILITY], lowBound=0, cat='Continuous')
Ypq_var = LpVariable.dicts(PRIMARY_TO_SECONDARY, [(p, q) for p in PRIMARY_FACILITY for q in SECONDARY_FACILITY], lowBound=0, cat='Continuous')
Zqk_var = LpVariable.dicts(SECONDARY_TO_CUSTOMER, [(q, k) for q in SECONDARY_FACILITY for k in CUSTOMER], lowBound=0, cat='Continuous')
Tpp_var = LpVariable.dicts(PRIMARY_TO_PRIMARY, [(p1, p2) for p1 in PRIMARY_FACILITY for p2 in set(PRIMARY_FACILITY)-set([p1])], lowBound=0, cat='Continuous')

G_var = {('p', p): LpVariable(name=FACILITY + '_p_' + str(p), cat='Binary') for p in PRIMARY_FACILITY}
G_var.update({('q', q): LpVariable(name=FACILITY + '_q_' + str(q), cat='Binary') for q in SECONDARY_FACILITY})


In [10]:
## Add constraints for R_P and R_Q
model += lpSum(G_var[('p',p)] for p in PRIMARY_FACILITY) == R_PRIMARY
model += lpSum(G_var[('q',q)] for q in SECONDARY_FACILITY) == R_SECONDARY


In [11]:
for p in PRIMARY_FACILITY:
    model += lpSum(Xsp_var[(s, p)] for s in SUPPLIER) <= lpSum(DEMAND_DICT[k] * G_var[('p', p)] for k in CUSTOMER)
    model += lpSum(Ypq_var[(p, q)] for q in SECONDARY_FACILITY) <= lpSum(DEMAND_DICT[k] * G_var[('p', p)] for k in CUSTOMER)

for q in SECONDARY_FACILITY:
    model += lpSum(Ypq_var[(p, q)] for p in PRIMARY_FACILITY) <= lpSum(DEMAND_DICT[k] * G_var[('q', q)] for k in CUSTOMER)
    model += lpSum(Zqk_var[(q, k)] for k in CUSTOMER) <= lpSum(DEMAND_DICT[k] * G_var[('q', q)] for k in CUSTOMER)


In [12]:
for p in PRIMARY_FACILITY:
    model += lpSum(Tpp_var[(p, p_)] for p_ in set(PRIMARY_FACILITY)-set([p])) <= lpSum(DEMAND_DICT[k] * G_var[('p', p)] for k in CUSTOMER)

for p_ in PRIMARY_FACILITY:
    model += lpSum(Tpp_var[(p, p_)] for p in set(PRIMARY_FACILITY)-set([p_])) <= lpSum(DEMAND_DICT[k] * G_var[('p', p_)] for k in CUSTOMER)


In [13]:
for p in PRIMARY_FACILITY:
    model += lpSum(Xsp_var[(s, p)] for s in SUPPLIER) + lpSum(Tpp_var[(p_, p)] for p_ in set(PRIMARY_FACILITY)-set([p])) == lpSum(Ypq_var[(p, q)] for q in SECONDARY_FACILITY) + lpSum(Tpp_var[(p, p_)] for p_ in set(PRIMARY_FACILITY)-set([p]))


In [14]:
for q in SECONDARY_FACILITY:
    model += lpSum(Ypq_var[(p, q)] for p in PRIMARY_FACILITY) == lpSum(Zqk_var[(q, k)] for k in CUSTOMER)


In [15]:
for p in PRIMARY_FACILITY:
    model += lpSum(Tpp_var[(p_, p)] for p_ in set(PRIMARY_FACILITY)-set([p])) == L * lpSum(Ypq_var[(p, q)] for q in SECONDARY_FACILITY)


In [16]:
for k in CUSTOMER:
    model += lpSum(Zqk_var[(q, k)] for q in SECONDARY_FACILITY) == DEMAND_DICT[k]


In [17]:
model.sense = LpMinimize

In [18]:
objective = lpSum(C_SUPPLIER * DISTANCE_DICT['SP'][f'XSP_{s[1:]}_{p[1:]}'] * Xsp_var[(s, p)] for s in SUPPLIER for p in PRIMARY_FACILITY) + \
            lpSum(C_PRIMARY * DISTANCE_DICT['PQ'][f'XPQ_{p[1:]}_{q[1:]}'] * Ypq_var[(p, q)] for p in PRIMARY_FACILITY for q in SECONDARY_FACILITY) + \
            lpSum(C_PRIMARY * DISTANCE_DICT['PP_'][f'XPP_{p[1:]}_{p_[1:]}'] * Tpp_var[(p, p_)] for p in PRIMARY_FACILITY for p_ in set(PRIMARY_FACILITY)-set([p])) + \
            lpSum(C_CUSTOMER * DISTANCE_DICT['QK'][f'XQK_{q[1:]}_{k[1:]}'] * Zqk_var[(q, k)] for q in SECONDARY_FACILITY for k in CUSTOMER) + \
            lpSum(FIXED_COST_PRIMARY * G_var[('p', p)] for p in PRIMARY_FACILITY) +\
            lpSum(FIXED_COST_SECONDARY * G_var[('q', q)] for q in SECONDARY_FACILITY) + \
            lpSum(PACKAGING_COST_PRIMARY * (lpSum(Ypq_var[(p, q)] for q in SECONDARY_FACILITY) + lpSum(Tpp_var[(p, p_)] for p_ in set(PRIMARY_FACILITY)-set([p]))) for p in PRIMARY_FACILITY) + \
            lpSum(PACKAGING_COST_SECONDARY * (lpSum(Zqk_var[(q, k)] for k in CUSTOMER)) for q in SECONDARY_FACILITY)


In [19]:
model.setObjective(objective)

In [20]:
LpProblem.writeLP(model, '../pulp_exps/MILP_pulp.lp')


[G_p_P0,
 G_p_P1,
 G_p_P10,
 G_p_P11,
 G_p_P12,
 G_p_P13,
 G_p_P14,
 G_p_P15,
 G_p_P16,
 G_p_P17,
 G_p_P18,
 G_p_P19,
 G_p_P2,
 G_p_P20,
 G_p_P21,
 G_p_P22,
 G_p_P23,
 G_p_P24,
 G_p_P25,
 G_p_P26,
 G_p_P27,
 G_p_P28,
 G_p_P29,
 G_p_P3,
 G_p_P30,
 G_p_P31,
 G_p_P32,
 G_p_P33,
 G_p_P34,
 G_p_P35,
 G_p_P36,
 G_p_P37,
 G_p_P38,
 G_p_P39,
 G_p_P4,
 G_p_P40,
 G_p_P41,
 G_p_P42,
 G_p_P43,
 G_p_P44,
 G_p_P45,
 G_p_P46,
 G_p_P47,
 G_p_P48,
 G_p_P49,
 G_p_P5,
 G_p_P6,
 G_p_P7,
 G_p_P8,
 G_p_P9,
 G_q_Q0,
 G_q_Q1,
 G_q_Q10,
 G_q_Q100,
 G_q_Q101,
 G_q_Q102,
 G_q_Q103,
 G_q_Q104,
 G_q_Q105,
 G_q_Q106,
 G_q_Q107,
 G_q_Q108,
 G_q_Q109,
 G_q_Q11,
 G_q_Q110,
 G_q_Q111,
 G_q_Q112,
 G_q_Q113,
 G_q_Q114,
 G_q_Q115,
 G_q_Q116,
 G_q_Q117,
 G_q_Q118,
 G_q_Q119,
 G_q_Q12,
 G_q_Q120,
 G_q_Q121,
 G_q_Q122,
 G_q_Q123,
 G_q_Q124,
 G_q_Q125,
 G_q_Q126,
 G_q_Q127,
 G_q_Q128,
 G_q_Q129,
 G_q_Q13,
 G_q_Q130,
 G_q_Q131,
 G_q_Q132,
 G_q_Q133,
 G_q_Q134,
 G_q_Q135,
 G_q_Q136,
 G_q_Q137,
 G_q_Q138,
 G_q_Q139,
 G_q_Q14,
 G

In [21]:
# # Get the number of variables
# num_variables = len(model.getVars())

# # Get the number of constraints
# num_constraints = len(model.getConstrs())

num_variables = len(model.variables())
num_constraints = len(model.constraints)

print(f'# Var : {num_variables}')
print(f'# Constraints : {num_constraints}')

# Var : 213550
# Constraints : 2173


In [22]:
model.status

0

In [23]:
LpSolverDefault.msg = 1

In [24]:
# model.optimize()
model.solve(PULP_CBC_CMD(msg=1, timeLimit=1000))


1

In [25]:
for p in range(50):
    if pulp.value(Xsp_var['S0', 'P'+str(p)])!=0:
        print(p, pulp.value(Xsp_var['S0', 'P'+str(p)]))
        

33 687133.0
44 687133.0


In [26]:
for p in range(50):
    for q in range(500):
        if pulp.value(Ypq_var[f'P{p}', f'Q{q}']) != 0:
            print(p, q, pulp.value(Ypq_var[f'P{p}', f'Q{q}']))


33 14 7935.0
33 17 27269.0
33 59 19490.0
33 113 15582.0
33 126 27201.0
33 152 20448.0
33 153 86772.0
33 155 14306.0
33 201 32362.0
33 210 34972.0
33 232 49635.0
33 264 2705.0
33 282 33770.0
33 289 22838.0
33 298 18576.0
33 306 25926.0
33 307 101572.0
33 321 73409.0
33 350 35198.0
33 351 11706.0
33 361 5530.0
33 389 43243.0
33 440 1505.0
33 442 24827.0
33 443 6750.0
33 469 23371.0
33 472 5969.0
33 475 18420.0
33 476 19155.0
44 103 108992.0
44 129 26183.0
44 133 15551.0
44 139 8741.0
44 151 31395.0
44 159 19983.0
44 194 30284.0
44 197 39062.0
44 224 36799.0
44 251 43092.0
44 275 2557.0
44 331 23838.0
44 346 27596.0
44 372 25073.0
44 388 16364.0
44 428 27395.0
44 441 20708.0
44 448 5172.0
44 462 15806.0
44 474 4891.0
44 482 34342.0


In [27]:
for q in range(500):
    for k in range(371):
        if pulp.value(Zqk_var[f'Q{q}', f'K{k}']) != 0:
            print(q, k, pulp.value(Zqk_var[f'Q{q}', f'K{k}']))


14 62 4256.0
14 73 3679.0
17 29 11869.0
17 77 3255.0
17 103 2342.0
17 141 1820.0
17 158 1659.0
17 159 1738.0
17 162 1619.0
17 189 1540.0
17 240 1004.0
17 305 423.0
59 66 3603.0
59 70 2947.0
59 83 2466.0
59 89 2549.0
59 121 2138.0
59 124 1701.0
59 144 1767.0
59 212 1125.0
59 213 1194.0
103 1 71683.0
103 23 14691.0
103 26 12540.0
103 32 10078.0
113 40 7286.0
113 145 1466.0
113 181 1152.0
113 236 903.0
113 244 891.0
113 256 731.0
113 263 589.0
113 276 628.0
113 280 489.0
113 303 326.0
113 312 290.0
113 321 203.0
113 324 233.0
113 326 142.0
113 336 70.0
113 340 72.0
113 342 81.0
113 354 15.0
113 355 15.0
126 27 12406.0
126 96 2488.0
126 112 2205.0
126 122 2176.0
126 125 2120.0
126 169 1432.0
126 173 1663.0
126 174 1437.0
126 191 1274.0
129 20 13818.0
129 74 3149.0
129 98 2332.0
129 149 1740.0
129 164 1795.0
129 187 1325.0
129 209 1340.0
129 275 684.0
133 16 15551.0
139 148 1544.0
139 161 1456.0
139 183 956.0
139 201 1178.0
139 222 942.0
139 243 750.0
139 249 659.0
139 255 667.0
139 320 141

In [28]:
for p in range(50):
    for p_ in range(50):
        if p!=p_:
            if pulp.value(Tpp_var[f'P{p}', f'P{p_}']) != 0:
                print(p, p_, pulp.value(Tpp_var[f'P{p}', f'P{p_}']))


33 44 281912.0
44 33 405221.0
